In [1]:
import onnx
import onnxruntime as ort
import cv2
import tensorflow as tf
import numpy as np
import time

In [3]:
session = ort.InferenceSession('models/onnxvgg16.onnx', None)
input_name = session.get_inputs()[0].name
output_thresh = session.get_outputs()[0].name
output_coords = session.get_outputs()[1].name

In [6]:
cap = cv2.VideoCapture(0)
num_frames = 0
start_time = time.time()
total_inference_time = 0
while cap.isOpened():
    _ , frame = cap.read()
    frame = frame[50:500, 50:500,:]

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120,120))
    img = np.expand_dims(resized/255,0)
    img1 = np.expand_dims(resized/255,0) 
    time1 = time.time()
    threshold = session.run([output_thresh], {input_name: img})
    coords = session.run([output_coords], {input_name: img1})
    time2 = time.time()
    time3 = time2 - time1
    total_inference_time += time3  # Add current inference time to total inference time
    print(f"Inference time: {time3:.3f} s")
    output_threshh = np.array(threshold)
    output_coord = np.array(coords[0][0])
    yhat = output_threshh, output_coords

    
    if yhat[0] > 0.8: 
        # Controls the main rectangle
        cv2.rectangle(frame, 
                      tuple(np.multiply(output_coord[:2], [450,450]).astype(int)),
                      tuple(np.multiply(output_coord[2:], [450,450]).astype(int)), 
                            (255,0,0), 2)
        # Controls the label rectangle
        cv2.rectangle(frame, 
                      tuple(np.add(np.multiply(output_coord[:2], [450,450]).astype(int), 
                                    [0,-30])),
                      tuple(np.add(np.multiply(output_coord[:2], [450,450]).astype(int),
                                    [80,0])), 
                            (255,0,0), -1)
        
        # Controls the text rendered
        cv2.putText(frame, 'face', tuple(np.add(np.multiply(output_coord[:2], [450,450]).astype(int),
                                               [0,-5])),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    cv2.imshow('EyeTrack', frame)

    num_frames += 1
    fps = num_frames / (time.time() - start_time)
    print(f'Frame rate: {fps:.2f} fps')
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

avg_inference_time = total_inference_time / num_frames
print(f'Total number of frames: {num_frames}')
print(f'Total elapsed time: {time.time() - start_time:.2f} s')
print(f'Average FPS: {num_frames / (time.time() - start_time):.2f} fps')
print(f'Average inference time: {avg_inference_time:.2f} s')


Inference time: 0.051 s
Frame rate: 1.54 fps
Inference time: 0.055 s
Frame rate: 2.77 fps
Inference time: 0.030 s
Frame rate: 3.75 fps
Inference time: 0.034 s
Frame rate: 4.72 fps
Inference time: 0.054 s
Frame rate: 5.39 fps
Inference time: 0.034 s
Frame rate: 6.17 fps
Inference time: 0.034 s
Frame rate: 6.87 fps
Inference time: 0.034 s
Frame rate: 7.44 fps
Inference time: 0.030 s
Frame rate: 7.99 fps
Inference time: 0.036 s
Frame rate: 8.47 fps
Inference time: 0.032 s
Frame rate: 8.88 fps
Inference time: 0.028 s
Frame rate: 9.35 fps
Inference time: 0.032 s
Frame rate: 9.66 fps
Inference time: 0.053 s
Frame rate: 9.87 fps
Inference time: 0.051 s
Frame rate: 10.13 fps
Inference time: 0.052 s
Frame rate: 10.26 fps
Inference time: 0.029 s
Frame rate: 10.54 fps
Inference time: 0.052 s
Frame rate: 10.69 fps
Inference time: 0.033 s
Frame rate: 10.87 fps
Inference time: 0.052 s
Frame rate: 10.96 fps
Inference time: 0.028 s
Frame rate: 11.27 fps
Inference time: 0.047 s
Frame rate: 11.39 fps
In

In [36]:
import onnx

# Load the ONNX model
model_path = "models/model_fp16.onnx"
model = onnx.load(model_path)

# Iterate over the nodes in the model's graph
for node in model.graph.node:
    # Print the node name, input shapes, and output shapes
    print(f"Node: {node.name}")
    


Node: model_1/vgg16/block1_conv1/BiasAdd__8
Node: model_1/vgg16/block1_conv1/BiasAdd
Node: model_1/vgg16/block1_conv1/Relu
Node: model_1/vgg16/block1_conv2/BiasAdd
Node: model_1/vgg16/block1_conv2/Relu
Node: model_1/vgg16/block1_pool/MaxPool
Node: model_1/vgg16/block2_conv1/BiasAdd
Node: model_1/vgg16/block2_conv1/Relu
Node: model_1/vgg16/block2_conv2/BiasAdd
Node: model_1/vgg16/block2_conv2/Relu
Node: model_1/vgg16/block2_pool/MaxPool
Node: model_1/vgg16/block3_conv1/BiasAdd
Node: model_1/vgg16/block3_conv1/Relu
Node: model_1/vgg16/block3_conv2/BiasAdd
Node: model_1/vgg16/block3_conv2/Relu
Node: model_1/vgg16/block3_conv3/BiasAdd
Node: model_1/vgg16/block3_conv3/Relu
Node: model_1/vgg16/block3_pool/MaxPool
Node: model_1/vgg16/block4_conv1/BiasAdd
Node: model_1/vgg16/block4_conv1/Relu
Node: model_1/vgg16/block4_conv2/BiasAdd
Node: model_1/vgg16/block4_conv2/Relu
Node: model_1/vgg16/block4_conv3/BiasAdd
Node: model_1/vgg16/block4_conv3/Relu
Node: model_1/vgg16/block4_pool/MaxPool
Node: 

In [38]:
import onnx
from onnx.shape_inference import infer_shapes

# Load the ONNX model
model_path = "models/model_fp16.onnx"
model = onnx.load(model_path)

# Infer shapes for all tensors in the model
model = infer_shapes(model)

# Iterate over the nodes in the model's graph
for node in model.graph.node:
    # Print the node name, input shapes, and output shapes
    print(f"Node: {node.name}")
    print(f"  Inputs: {[input_tensor.type.tensor_type.shape.dim for input_tensor in node.input_tensor_values]}")
    print(f"  Outputs: {[output_tensor.type.tensor_type.shape.dim for output_tensor in node.output_tensor_values]}")

    # Print the precision and quantization values for each output
    for i, output_tensor in enumerate(node.output):
        print(f"  Output {i}:")
        tensor_value_info = next((t for t in model.graph.value_info if t.name == output_tensor), None)
        if tensor_value_info is not None and tensor_value_info.type.tensor_type is not None:
            tensor_type = tensor_value_info.type.tensor_type
            if tensor_type.elem_type == onnx.TensorProto.FLOAT:
                print(f"    Precision: float32")
            elif tensor_type.elem_type == onnx.TensorProto.INT8:
                print(f"    Precision: int8")
                if tensor_type.quantization is not None:
                    print(f"    Scale: {tensor_type.quantization.scale}")
                    print(f"    Zero point: {tensor_type.quantization.zero_point}")
            elif tensor_type.elem_type == onnx.TensorProto.UINT8:
                print(f"    Precision: uint8")
                if tensor_type.quantization is not None:
                    print(f"    Scale: {tensor_type.quantization.scale}")
                    print(f"    Zero point: {tensor_type.quantization.zero_point}")
            else:
                print(f"    Precision: unknown")


Node: model_1/vgg16/block1_conv1/BiasAdd__8


AttributeError: input_values